# imports and data loading

In [1]:
%run Imports.ipynb
name = 'Kred'

# specific imports
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV


key_featsubgroups = pd.read_pickle('../../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../../pickle/2_FS/' + name + '/2_df_new_.pkl')

c:\Users\benjf\anaconda3\envs\Work\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [3]:
# param grid scikit
param_grid = {
    'hidden_layer_sizes': [(h,) for h in [5, 10, 20, 50, 100, 200, 300, 406]],
    'activation': ['relu', 'tanh', 'logistic'],                   # activation functions
    'solver': ['adam', 'sgd'],                                    # optimization algorithms
    'alpha': [0.0001, 0.001, 0.01, 0.1],                                # L2 regularization
    'learning_rate': ['constant', 'adaptive'],                    # learning rate strategy
    'learning_rate_init': [0.001, 0.005, 0.01, 0.05],                           # initial learning rate
    'max_iter': [100, 200, 500, 750, 1000]                                        # training epochs
}

In [4]:
with open('../../pickle/4_Model_Optuna/results_dict_updated.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [5]:
with open('../../pickle/3_Model/xgb_default_params.pkl', 'rb') as f:
    default_params = pickle.load(f)


# data splitting

In [ ]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = split_data_4(df)

# Task 2

In [ ]:
# choosing optimal parameters
clf_best = MLPClassifier(**best_params, random_state=0)
clf_best.fit(x_train, y_train)

In [ ]:
# Predict test data
y_test_pred  = clf_best.predict(X_test)
y_test_proba = clf_best.predict_proba(X_test)[:, 1]

In [ ]:
# Extract feature importance score from trained model using permutation-based feature importance scoring
from sklearn.inspection import permutation_importance

perm_imp = permutation_importance(
    clf_best, X_test, y_test,
    n_repeats=30, random_state=0, n_jobs=-1
)
feature_importances = perm_imp.importances_mean

In [ ]:
# Store test set predictions and feature importance scores

results_M7_T2 = {
    "y_test_pred":         y_test_pred,
    "y_test_proba":        y_test_proba,
    "feature_importances": feature_importances
}

# please update the path as needed
with open("results_M7_T2.pkl", "wb") as f:
    pickle.dump(results_M7_T2, f, protocol=pickle.HIGHEST_PROTOCOL)